In [2]:
import tensorflow as tf
import numpy as np

In [3]:
# 数据获取及预处理
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        print(self.train_data.shape)
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        print(self.train_data.shape)
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]

In [4]:
num_epochs = 1
batch_size = 100
learning_rate = 0.001

inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(units=10)(x)
outputs = tf.keras.layers.Softmax()(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

(60000, 28, 28)
(60000, 28, 28, 1)


In [5]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.448332
batch 1: loss 2.232373
batch 2: loss 2.217708
batch 3: loss 2.140573
batch 4: loss 2.077614
batch 5: loss 1.974723
batch 6: loss 1.873117
batch 7: loss 1.873246
batch 8: loss 1.823415
batch 9: loss 1.793207
batch 10: loss 1.711310
batch 11: loss 1.512110
batch 12: loss 1.575150
batch 13: loss 1.484524
batch 14: loss 1.410708
batch 15: loss 1.418751
batch 16: loss 1.318666
batch 17: loss 1.289241
batch 18: loss 1.268923
batch 19: loss 1.182100
batch 20: loss 1.138317
batch 21: loss 1.042774
batch 22: loss 1.024451
batch 23: loss 1.010646
batch 24: loss 0.887569
batch 25: loss 0.953281
batch 26: loss 0.863570
batch 27: loss 0.984607
batch 28: loss 0.768714
batch 29: loss 0.820731
batch 30: loss 0.861115
batch 31: loss 0.827887
batch 32: loss 0.709560
batch 33: loss 0.617104
batch 34: loss 0.657366
batch 35: loss 0.692014
batch 36: loss 0.801132
batch 37: loss 0.620606
batch 38: loss 0.647980
batch 39: loss 0.609436
batch 40: loss 0.566316
batch 41: loss 0.749932
ba

batch 339: loss 0.299434
batch 340: loss 0.183660
batch 341: loss 0.170845
batch 342: loss 0.215946
batch 343: loss 0.286053
batch 344: loss 0.257184
batch 345: loss 0.244548
batch 346: loss 0.307304
batch 347: loss 0.178251
batch 348: loss 0.190371
batch 349: loss 0.186960
batch 350: loss 0.212605
batch 351: loss 0.202421
batch 352: loss 0.169142
batch 353: loss 0.237986
batch 354: loss 0.130283
batch 355: loss 0.259669
batch 356: loss 0.154367
batch 357: loss 0.248631
batch 358: loss 0.182629
batch 359: loss 0.361344
batch 360: loss 0.210905
batch 361: loss 0.265098
batch 362: loss 0.272234
batch 363: loss 0.197086
batch 364: loss 0.117507
batch 365: loss 0.190136
batch 366: loss 0.354583
batch 367: loss 0.244001
batch 368: loss 0.226531
batch 369: loss 0.281699
batch 370: loss 0.311118
batch 371: loss 0.336741
batch 372: loss 0.199976
batch 373: loss 0.288680
batch 374: loss 0.166038
batch 375: loss 0.190006
batch 376: loss 0.175793
batch 377: loss 0.139932
batch 378: loss 0.291172


In [6]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.943500
